# This notebook showcases the basic usage of RNNs to make predictions on time series data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

### Generating sin curve data

In [ ]:
N = 1000    
Tp = 800    

t=np.arange(0,N)
x=np.sin(0.02*t)+2*np.random.rand(N)
df = pd.DataFrame(x)
plt.plot(df)
plt.show() 

### Reshape to represent sequence prediction

Examples:

X = [1,2,3,4], Y = [5]

X = [2,3,4,5], Y = [6]

In [ ]:
values=df.values
# Splitting values into train (800 points) and test (200 points)
# Note: Data is still in it's original form
train,test = values[0:Tp,:], values[Tp:N,:]
print(train.shape)
print(test.shape)

In [ ]:
# What is the actual X shape needed to represent the above examples?

In [ ]:
step = 4
test = np.append(test,np.repeat(test[-1,],step))
train = np.append(train,np.repeat(train[-1,],step))

print(test.shape)
print(train.shape)

In [ ]:
tmp = np.array([[1],[2],[3],[4]])
tmp.shape

In [ ]:
tmp = np.append(tmp, np.repeat(tmp[-1,],2))
tmp

In [ ]:
# convert into dataset matrix
def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

In [ ]:
trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

trainX.shape

In [ ]:
print("Shape of train data after appended repeats:",train.shape)
print("First 4 elements in train data:", train[0],train[1],train[3],train[4])
print("Shape of converted train data:",trainX.shape)
print("Value of first element in converted train data:",trainX[0])
print("Value of first element in train label:", trainY[0])

### Build model

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=32, input_shape=(1,step), activation="relu"))
model.add(Dense(8, activation="relu")) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

In [ ]:
model.fit(trainX,trainY, epochs=100, batch_size=16, verbose=2)
trainPredict = model.predict(trainX)
testPredict= model.predict(testX)

In [ ]:
plt.plot(df[:800])
plt.plot(trainPredict)